<img align="left" src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=100 style="padding: 20px"> 


<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=160 style="padding: 20px">  

## Photo-z Server - Tutorial Notebook

**Contact author**: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br)) 

**Last verified run**: yyyy-mm-dd <br>



### Introduction
The Photo-z Server is an online service available for the LSST Community to host and share lightweight photo-z related data products. The upload and download of data and metadata is done at the website www.linea.org.br/pz-server (RSP authentication required). There you will find two separate pages containing a list of data products each: one for LSST Data Management's oficial data products, and one for user-generated data products. 
<font color=red> to do: update URL when production environment is online </font>

The registered data products can also be accessed directly from Python code using the Photo-z Server's data access API, as demonstrated below. For a comprehensive documentation about the Photo-z Server, please visit the [Photo-z Server's documentation page](<add link to readthedocs>).
<font color=red> to do: add link to readthedocs  </font>

The Photo-z Server is developed and delivered as part of the in-kind contribution program BRA-LIN, from LIneA to the Rubin Observatory's LSST. An overview of this and other contributions related to Photo-zs is available [here](https://linea-it.github.io/pz-lsst-inkind-doc/).  


#### Setup

In [ ]:
! export PYTHONPATH=$PYTHONPATH:/Users/julia/github/pz-server-lib

In [ ]:
from pz_server import PzServer

The connection with the Photo-z Server from Python code is done by an object of the class `PzServer`. To get authorization to define an instance of `PzServer`, the RSP users must provide an **API Token** generated on the [Photo-z Server website](www.linea.org.br/pz-server). See the steps on the screenshots below.  
<font color=red> to do: check link URL </font>

<img src="./images/ScreenShot1.png" width=110pt style="padding: 30px"/> <img src="./images/ScreenShot2.png" width=300pt />


<font color=red> to do: UPDATE screenshots  </font>

In [ ]:
pz_server = PzServer(token="<paste your API Token here>") 

#### Get basic info from PZ Server

The object `pz_server` brings some useful attributes for the user to navigate throug the available contents.

For instance:

Display the list of product types supported with a short description;

In [ ]:
pz_server.list_product_types.to_pandas() 

Display the list of users who uploaded data products to the server;

In [ ]:
pz_server.list_users.to_pandas()

Display the list of data releases available at the time. 

In [ ]:
pz_server.list_releases.to_pandas() 

The information above can be used to filter the data products of interest for your search.

The cells below show examples of usage organized by product types. 

## Spec-z Catalog 
#### List Spec-z Catalogs available on Pz Server

In [ ]:
pz_server.list_products(filters={user:"PZ Coord. Group",  
                                 product_type="specz_catalog", 
                                 release="all"}).to_pandas()

#### Get metadata of a given Spec-z Catalog 

In [ ]:
metadata_specz_catalog = pz_server.get_product_metadata(product_id="0001")
metadata_specz_catalog 

#### Retrieve a given Spec-z Catalog (as Astropy table or Pandas DataFrame)

In [ ]:
specz_catalog = pz_server.get_product(product_id="0001", save_file="False").to_pandas()
specz_catalog

Display basic statistics

In [ ]:
dataframe_specz_1.describe()

Quick visualization of spec-z catalog properties

In [ ]:
pz_server.specz_plots(data=specz_catalog, metadata=metadata_specz_catalog)

## Training and Validation Sets 
#### List Training and Validation Sets available on Pz Server

In [ ]:
pz_server.list_products(filters={user:"PZ Coord. Group",   
                                 product_type="train_set", 
                                 release="all"}).to_pandas()

In [ ]:
pz_server.list_products(filters={user:"PZ Coord. Group",   
                                 product_type="valid_set", 
                                 release="all"}).to_pandas()

#### Get metadata of a given Training Set

In [ ]:
metadata_train_set = PzServer.get_product_metadata(product_id="0004")
metadata_train_set 

#### Get metadata of a given Validation Set

In [ ]:
metadata_valid_set = PzServer.get_product_metadata(product_id="0005")
metadata_valid_set 

#### Retrieve a given Training (or Validation) Set (as Astropy table or Pandas DataFrame)

In [ ]:
train_set = PzServer.get_product(product_id="0004", save_file="False").to_pandas()
valid_set = PzServer.get_product(product_id="0005", save_file="False").to_pandas()

Display basic statistics

In [ ]:
dataframe_train_set.describe()

In [ ]:
dataframe_valid_set.describe()

Quick visualization of training/validation sets properties

In [ ]:
pz_server.train_valid_plots(data=[dataframe_train_set, dataframe_valid_set], 
                         metadata=[metadata_train_set, metadata_valid_set])

## Photo-z Validation Results
#### List Photo-z Validation Results available on Pz Server

In [ ]:
pz_server.list_products(filters={user:"PZ Coord. Group",   
                                 product_type="pz_result", 
                                 release="all"}).to_pandas()
                        # default: filter=None -> show all products 
                        # return dic with product_id an short description provided by the user when uploaded

#### Get metadata of a given data product of Photo-z Validation Results

In [ ]:
metadata_pz_result = PzServer.get_product_metadata(product_id="0006")
metadata_pz_result  # markdown table

#### Retrieve a given Photo-z Validation Result: download .tar file

This product type is not necessarily (only) tabular data and can be a list of files, so the function `get_produt` will not return an Astropy table. Instead, it will return the name of the tar file downloaded to the local directory. 

In [ ]:
pz_result = PzServer.get_product(product_id="0006", save_file="True")
pz_result # string tar file name

#### Basic Photo-z Validation Plots

Reuse RAIL evaluation plots and tables for illustration with an example of Photo-z Validation Results. 

In [ ]:
#  open .tar file 

In [ ]:
from RAIL.examples.evaluation.utils import *
from RAIL.rail.evaluation.metrics.pit import *
from RAIL.rail.evaluation.metrics.cdeloss import *

In [ ]:
my_path = 'xxx/xxx/xx' 
pdfs_file =  os.path.join(my_path, "pdfs_FZBoost.hdf5")
ztrue_file =  os.path.join(my_path, "ztrue_validation_set.hdf5")
pdfs, zgrid, ztrue, photoz_mode = read_pz_output(pdfs_file, ztrue_file) # all numpy arrays

Plot PIT-QQ

In [ ]:
plot_pit_qq(pdfs, zgrid, ztrue, title="PIT-QQ - toy data", code="FZBoost",
                pit_out_rate=pit_out_rate, savefig=False)

PZ Validation metrics table

In [ ]:
summary = Summary(pdfs, zgrid, ztrue)
summary.markdown_metrics_table(pitobj=pitobj) # pitobj as optional input to speed-up metrics evaluation